# CTCF / SIX5 / ZNF143 Data Analysis

Here’s an idea: we can try to replicate the test using the raw number of bases that overlap between the files. That would be as close to the original data as we can get, and it’s something Anshul would have the time to replicate. Try setting up a single notebook that does this analysis:

The total number of bases on the assembled human chromosomes is 3095677412 - you get this by summing up columns chr1-chr21 here: http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.chrom.sizes. We can let 3095677412 represent the size of our “universe”. Ignore ambiguous regions for now - it would be surprising if they altered the overall trend.
(1) The total number of bases covered by idr optimal CTCF peaks are the “special” set - count those up and print out the number in a cell. Good to put the CTCF peaks through `bedtools merge -i ctcf_file.bed > merged_ctcf_file.bed` before you do this to avoid double-counting bases in overlapping regions.
(2) The total number of bases covered by idr optimal SIX5 peaks are the “picked” set - count those up and print out the number in a cell. Again, good to put the SIX5 peaks through `bedtools merge` to avoid double-counting bases.
(3) The total number of bases in the intersection of CTCF and SIX5 peaks are the “special, picked” set. Find those bases by counting up the number of bases in the file produced by `bedtools intersect -a ctcf_file.bed -b six5_file.bed | bedtools merge > merged_ctcf_six5_intersection.bed`
(4) Print out the ratio (special,picked)/(special,not_picked) / (not_special,picked)/(not_special,not_picked). Also print out the p-value using the fisher’s exact test.
(5) Repeat the analysis above, but conditioning on bases that overlap the ZNF143 peak set.

In the meantime, I will let Anshul know the results of this analysis to get his initial thoughts.

In [1]:
universe = 3095677412
data_dir = "/home/ktian/kundajelab/tfnet/ENCODE_data/"
#data_dir = "/home/ktian/oak/stanford/groups/akundaje/marinovg/TF-models/2018-06-15-ENCODE-TF-ChIP-seq-for-multitask-models/"

ctcf_file   = data_dir + "GM12878-CTCF-human-ENCSR000AKB-optimal_idr.narrowPeak.gz"
six5_file   = data_dir + "GM12878-SIX5-human-ENCSR000BJE-optimal_idr.narrowPeak.gz"
znf143_file = data_dir + "GM12878-ZNF143-human-ENCSR000DZL-optimal_idr.narrowPeak.gz"

import os

os.system("ls -l " + ctcf_file)

os.system("mkdir -p _tmp_")
merged_ctcf   = "_tmp_/merged_ctcf_file.bed"
merged_six5   = "_tmp_/merged_six5_file.bed"
merged_znf143 = "_tmp_/merged_znf143_file.bed"

cmd = "gunzip -c " + ctcf_file   + " | bedtools sort | bedtools merge > " + merged_ctcf
print(cmd)
os.system("gunzip -c " + ctcf_file   + " | bedtools sort | bedtools merge > " + merged_ctcf)
os.system("gunzip -c " + six5_file   + " | bedtools sort | bedtools merge > " + merged_six5)
os.system("gunzip -c " + znf143_file + " | bedtools sort | bedtools merge > " + merged_znf143)

gunzip -c /home/ktian/kundajelab/tfnet/ENCODE_data/GM12878-CTCF-human-ENCSR000AKB-optimal_idr.narrowPeak.gz | bedtools sort | bedtools merge > _tmp_/merged_ctcf_file.bed


0

In [2]:
import pandas as pd
def sum_up_interval_len(file, desc):
    df = pd.read_csv(file, sep='\t', index_col=None, header=None)
    #experiments = ['ENCSR000AKB', 'ENCSR000BJE','ENCSR000DZL']
    #df.columns = ['chr', 'start', 'end']
    #print(df.shape)
    #print(df.head(5))
    
    df_new = df.iloc[:,1:].astype('int') # select all rows and all columns except the first one (IDs), convert to int

    df_len = df_new.iloc[:,1] - df_new.iloc[:,0] # end - start
    #print(df_len.head(5))
    
    total_len = df_len.sum()
    print("%12s: #bp in merged file is %10d" % (desc, total_len))
    return total_len

In [3]:
len_ctcf   = sum_up_interval_len(merged_ctcf, "ctcf")
len_six5   = sum_up_interval_len(merged_six5, "six5")
len_znf143 = sum_up_interval_len(merged_znf143, "znf143")

        ctcf: #bp in merged file is   12407139
        six5: #bp in merged file is    1180900
      znf143: #bp in merged file is   10903313


In [4]:
merged_ctcf_six5_intersection   = "_tmp_/merged_ctcf_six5_intersection.bed"
merged_six5_znf143_intersection = "_tmp_/merged_six5_znf143_intersection.bed"
merged_znf143_ctcf_intersection = "_tmp_/merged_znf143_ctcf_intersection.bed"
merged_all_intersection = "_tmp_/merged_all_intersection.bed"

os.system("bedtools intersect -a " + merged_ctcf   + " -b " + merged_six5 + " | bedtools merge > " + \
          merged_ctcf_six5_intersection)
os.system("bedtools intersect -a " + merged_six5   + " -b " + merged_znf143 + " | bedtools merge > " + \
          merged_six5_znf143_intersection)
os.system("bedtools intersect -a " + merged_znf143 + " -b " + merged_ctcf + " | bedtools merge > " + \
          merged_znf143_ctcf_intersection)
len_ctcf_six5   = sum_up_interval_len(merged_ctcf_six5_intersection,   "ctcf_six5")
len_six5_znf143 = sum_up_interval_len(merged_six5_znf143_intersection, "six5_znf143")
len_znf143_ctcf = sum_up_interval_len(merged_znf143_ctcf_intersection, "znf143_ctcf")


os.system("bedtools intersect -a " + merged_ctcf_six5_intersection + " -b " + merged_znf143 + " | bedtools merge > " + \
          merged_all_intersection)
len_all = sum_up_interval_len(merged_all_intersection, "three-way n")

   ctcf_six5: #bp in merged file is     130386
 six5_znf143: #bp in merged file is     729608
 znf143_ctcf: #bp in merged file is    4299081
 three-way n: #bp in merged file is     105690


In [5]:
def intersect_tfs(tf_a, tf_b):
    tfa_name = "_tmp_/merged_" + tf_a + "_file.bed"
    tfb_name = "_tmp_/merged_" + tf_b + "_file.bed"
    tf_ab_intersect = "_tmp_/merged_" + tf_a + "_" + tf_b + "_intersection.bed"
    cmd ="bedtools intersect -a _tmp_/merged_" + tf_a + "_file.bed -b _tmp_/merged_" + tf_b + \
         "_file.bed | bedtools merge > merged_" + tf_a + "_" + tf_b + "_intersection"
    #print(cmd)
    os.system(cmd)
    len_intersect = sum_up_interval_len("merged_" + tf_a + "_" + tf_b + "_intersection", tf_a + "_" + tf_b)
    return len_intersect

len_ctcf_six5   = intersect_tfs("ctcf", "six5")
len_six5_znf143 = intersect_tfs("six5", "znf143")
len_znf143_ctcf = intersect_tfs("znf143", "ctcf")
len_ctcf_znf143 = len_znf143_ctcf

   ctcf_six5: #bp in merged file is     130386
 six5_znf143: #bp in merged file is     729608
 znf143_ctcf: #bp in merged file is    4299081


(4) Print out the ratio (special,picked)/(special,not_picked) / (not_special,picked)/(not_special,not_picked). Also print out the p-value using the fisher’s exact test.
(5) Repeat the analysis above, but conditioning on bases that overlap the ZNF143 peak set.

In [6]:
#question 1: what is the coenrichment of CTCF with six5 (special = CTCF, picked = six5)
#contingency table should look like:

# [ pos_for_ctcf_and_six5, pos_for_ctcf_but_not_six5]
# [ neg_for_ctcf_pos_for_six5, neg_for_ctcf_and_six5]

from scipy.stats import fisher_exact
print(fisher_exact(table=[
    [len_ctcf_six5, len_ctcf - len_ctcf_six5],
    [len_six5 - len_ctcf_six5, universe - len_ctcf - len_six5 + len_ctcf_six5]
])) # PIE

#question 2: What is the coenrichment of CTCF with six5 in the presence of znf143
from scipy.stats import fisher_exact
print(fisher_exact(table=[
    [len_all, len_ctcf_znf143 - len_all],
    [len_six5_znf143 - len_all, len_znf143 - len_ctcf_znf143 - len_ctcf_znf143 + len_all]
])) # znf "universe"

(31.160843103361707, 0.0)
(0.09738892962553414, 0.0)
